In [9]:
config = {
    "base_url": 'https://api.chatfire.cn/v1',
    "api_key": 'sk-zO8exlBicZh7nJeZn5GuC5X9SPuVrZzXoGyOW0i9BFvN62ON'
}
import os,sys
os.environ["OPENAI_API_KEY"] = config['api_key']  # 替换成你的真实密钥
os.environ["OPENAI_BASE_URL"] = config['base_url']


In [10]:
from pydantic import BaseModel
import agents
from typing import List, Optional, Dict
import json

In [ ]:
# 用户故事agents
class UserStory(BaseModel):
    role: str
    goal: str
    value: str
    def to_dict(self):
        return {
            "role": self.role,
            "goal": self.goal,
            "value": self.value
        }

    def __str__(self):
        return json.dumps(self.to_dict(), ensure_ascii=False)

class StoryCollection(BaseModel):
    items: List[UserStory]

    def __str__(self):
        return json.dumps([item.to_dict() for item in self.items], indent=2, ensure_ascii=False)

@agents.function_tool
def get_roles():
    return "all roles : "\
        "customer,cashier,inventory manager,store manager,marketing manager," \
        "saler,delivery driver,customer service agent,accountant,auditor." \
        "besides, you can also define your own roles by your experiences."

use_story_agent = agents.Agent(
    name="User Story Generator",
    instructions="You are a project manager for a supermarket management system, "\
        "and you need to analyze and generate standard user stories based on actual scenarios.",
    output_type=StoryCollection,
    model="gpt-4o",
    tools=[get_roles]
)


In [ ]:
# 用例图agent
class UseCase(BaseModel):
    name: str
    include: List[str]
    extends: List[str]

    def to_dict(self):
        """转换为可序列化的字典结构"""
        return {
            "name": self.name,
                # 确保include和extends列表中的元素都转为字符串，以防万一
            "include": [inc for inc in self.include],
            "extends": [ext for ext in self.extends]
        }

    def __str__(self):
        return json.dumps(self.to_dict(), indent=2, ensure_ascii=False)

class UseCases(BaseModel):
    name: str
    usecases: List[UseCase]

    def to_dict(self):
        """完全转换为可序列化的字典结构"""
        return {
            "name": self.name,
            "usecases": [uc.to_dict() for uc in self.usecases]  # 确保每个用例都转为字典
        }
    
    def __str__(self):
        """JSON字符串表示"""
        return json.dumps(self.to_dict(), indent=2, ensure_ascii=False)
    


use_case_agent = agents.Agent(
    name="User Case Generator",
    instructions="You are a project manager for a supermarket management system,"\
        " and you need to generate some use case on format." \
        " you need to generate enough use case by input user stroy",
    output_type=UseCases,
    model="gpt-4o",
)

In [13]:
# 系统顺序图agent
class Participant(BaseModel):
    name: str
    type: str
    description: Optional[str]

    def to_dict(self):
        return {
            "name": self.name,
            "type": self.type,
            "description": self.description
        }
    
    def __str__(self):
        return json.dumps(self.to_dict(), indent=2, ensure_ascii=False)

class Interaction(BaseModel):
    sequence: int
    from_: str
    to: str
    message: str
    return_: Optional[str]

    def to_dict(self):
        return {
            "sequence": self.sequence,
            "from_": self.from_,
            "to": self.to,
            "message": self.message,
            "return_": self.return_
        }

    def __str__(self):
        return json.dumps(self.to_dict(), indent=2, ensure_ascii=False)


class SystemSequenceDiagram(BaseModel):
    title: str
    description: Optional[str]
    participants: List[Participant]
    interactions: List[Interaction]
    return_: Optional[str]

    def to_dict(self):
        return {
            "title": self.title,
            "description": self.description,
            "participants": [participant.to_dict() for participant in self.participants],
            "interactions": [interaction.to_dict() for interaction in self.interactions],
            "return_": self.return_
        }

    def __str__(self):
        return json.dumps(self.to_dict(), indent=2, ensure_ascii=False)

    def visualize(self):
        """模拟可视化输出"""
        print(f"=== {self.title} ===")
        for interaction in self.interactions:
            print(
                f"{interaction.sequence}. {interaction.from_} -> {interaction.to}: "
                f"{interaction.message} → {interaction.return_}"
            )

# 正确初始化Agent（移除output_schema参数）
sys_seq_diagram_agent = agents.Agent(
    name="SSD_Generator",
    instructions="You are a project manager for a supermarket management system, " \
        "and you need to generate some System Sequence Diagram on format." \
        " you need to generate a System Sequence Diagram based on the input use cases. ",
    output_type=SystemSequenceDiagram,
    model="gpt-4o"
)


In [ ]:
class ConceptClass(BaseModel):
    """表示概念类图中的类"""
    name: str
    attributes_names: List[str]
    attributes_types: List[str]
    # 属性名到类型的映射
    is_abstract: bool
    methods: List[str]
    # 在概念类图中通常为空

    def to_dict(self):
        return {
            'name': self.name,
            'attributes_names': self.attributes_names,
            'attributes_types': self.attributes_types,
            'is_abstract': self.is_abstract,
            'methods': self.methods
        }
    
    def __str__(self):
        return json.dumps(self.to_dict(), ensure_ascii=False, indent=2)

class Relationship(BaseModel):
    """表示类之间的关系"""
    source: str # 源类名
    target: str # 目标类名
    type: str # 关联、继承、聚合、组合等
    multiplicity: Optional[str] # 多重性，如 "1..*"
    label: Optional[str] # 关系标签
    def to_dict(self):
        return {
            'source': self.source,
            'target': self.target,
            'type': self.type,
            'multiplicity': self.multiplicity,
            'label': self.label
        }
    
    def __str__(self):
        return json.dumps(self.to_dict(), ensure_ascii=False, indent=2)

class ConceptClassDiagram(BaseModel):
    """完整的类图表示"""
    classes: List[ConceptClass]
    relationships: List[Relationship]
    def to_dict(self):
        return {
            'classes': [c.to_dict() for c in self.classes],
            'relationships': [r.to_dict() for r in self.relationships]
        }

    def __str__(self):
        return json.dumps(self.to_dict(), ensure_ascii=False, indent=2)



concept_class_diagram = agents.Agent(
    name="ConceptClassDiagram",
    instructions="You are a project manager for a supermarket management system" \
        " and you need to generate some concept class diagram on format." \
        " you need to refer input data and generate concept class diagram.",
    output_type=ConceptClassDiagram,
    model="gpt-4o"
)

In [15]:
# ocl合约生成agent
from enum import Enum

class OclConstraintType(str, Enum):
    INVARIANT = "invariant"
    PRECONDITION = "precondition"
    POSTCONDITION = "postcondition"
    GUARD = "guard"
    DERIVATION = "derivation"

class OclContextType(str, Enum):
    CLASS = "class"
    ATTRIBUTE = "attribute"
    OPERATION = "operation"
    ASSOCIATION = "association"

class OclExpression(BaseModel):
    """OCL 表达式基类"""
    body: str  # OCL 表达式字符串
    description: Optional[str] = None
    def to_dict(self):
        return {
            "body": self.body,
            "description": self.description
        }
    def __str__(self):
        return json.dumps(self.to_dict(), ensure_ascii=False, indent=2)

class OclContract(BaseModel):
    """OCL 合约基类"""
    name: str
    constraint_type: OclConstraintType
    context: str  # 应用约束的上下文（类名、方法名等）
    context_type: OclContextType
    expression: OclExpression
    severity: str = "error"  # error/warning/info
    is_static: bool = False  # 是否是静态约束
    def to_dict(self):
        return {
            "name": self.name,
            "constraint_type": self.constraint_type,
            "context": self.context,
            "context_type": self.context_type,
            "expression": self.expression.to_dict(),
            "severity": self.severity,
            "is_static": self.is_static
        }
    def __str__(self):
        return json.dumps(self.to_dict(), ensure_ascii=False, indent=2)

class OclContracts(BaseModel):
    """OCL 合约集合基类"""
    contracts: List[OclContract]
    def to_dict(self):
        return {
            "contracts": [contract.to_dict() for contract in self.contracts]
        }
    def __str__(self):
        return json.dumps(self.to_dict(), ensure_ascii=False, indent=2)

ocl_contracts_agents = agents.Agent(
    name="OclContracts",
    instructions="You are a project manager for a supermarket management system,"\
        " and you need to generate some ocl Contracts on format." \
        " you need to generate a list of OclContracts based on the following input Concept Class Diagram ",
    output_type=OclContracts,
    model="gpt-4o"
)






In [16]:
async def main():
    if not os.path.exists("output"):
        os.mkdir("output")

    stdout = sys.stdout

    with agents.trace("LLM Generator"):
        print("Hello! Software Engineering!")
        print("Now begin to generate user storys")
        print("========================begin")

        result = await agents.Runner.run(
            starting_agent=use_story_agent,
            input="generate many user storys,and obey the format of the output strictly",
        )

        input_item = result.to_input_list()

        sys.stdout = open("output/UserStory.json", "w")
        print(result.final_output)
        sys.stdout = stdout


        print("Now begin to generate user cases")

        result = await agents.Runner.run(
            starting_agent=use_case_agent,
            input=input_item,
        )
        input_item = result.to_input_list()

        sys.stdout = open("output/UseCase.json", "w")
        print(result.final_output)
        sys.stdout = stdout

        print("========================end")
        print("Now begin to generate sys seq diagrams")
        print("========================begin")
    

        result = await agents.Runner.run(
            starting_agent=sys_seq_diagram_agent,
            input=input_item,
            # input="generate many sys seq diagrams,and obey the format of the output strictly",
        )
        input_item = result.to_input_list()

        sys.stdout = open("output/SysSeqDiagram.json", "w")
        print(result.final_output)
        sys.stdout = stdout

        print("========================end")
        print("Now begin to generate concept class diagram")
        print("========================begin")

        result = await agents.Runner.run(
            starting_agent=concept_class_diagram,
            input=input_item,
            # input="generate many concept class diagram,and obey the format of the output strictly",
        )
        input_item = result.to_input_list()

        sys.stdout = open("output/ConceptClassDiagram.json", "w")
        print(result.final_output)
        sys.stdout = stdout
            
        result = await agents.Runner.run(
            starting_agent=ocl_contracts_agents,
            input=input_item,
            # input="generate many ocl contracts diagram,and obey the format of the output strictly",
        )
        sys.stdout = open("output/OclContracts.json", "w")
        print(result.final_output)
        sys.stdout = stdout

        print("========================end")


if __name__ == "__main__":
    # import asyncio
    # asyncio.run(main())  # 使用正确的asyncio运行方式
    await main()  # 使用正确的asyncio运行方式
    

Hello! Software Engineering!
Now begin to generate user storys
========================begin
Now begin to generate user cases
========================end
Now begin to generate sys seq diagrams
========================begin
========================end
Now begin to generate concept class diagram
========================begin
========================end
